In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Unicauca_20000.csv")
df.head()

,Unnamed: 0,Flow.ID,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,...,Active.Std,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName
0,3056380,179.1.4.223-10.200.7.194-443-60176-6,10.200.7.194,60176,179.1.4.223,443,6,15/05/201709:57:28,36,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,126,GOOGLE
1,1333820,192.168.10.8-10.200.7.6-54587-3128-6,192.168.10.8,54587,10.200.7.6,3128,6,27/04/201704:58:13,45155396,14,...,0.0,158289.0,158289.0,44997074.0,0.0,44997074.0,44997074.0,BENIGN,126,GOOGLE
2,2736896,179.1.4.231-10.200.7.217-80-58232-6,10.200.7.217,58232,179.1.4.231,80,6,11/05/201711:11:44,59513905,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,126,GOOGLE
3,2324344,10.200.7.217-64.233.190.188-39787-5228-6,10.200.7.217,39787,64.233.190.188,5228,6,11/05/201709:29:04,874682,16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,126,GOOGLE
4,3356613,179.1.4.238-10.200.7.199-80-58407-6,10.200.7.199,58407,179.1.4.238,80,6,15/05/201704:39:45,7532,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,126,GOOGLE


In [3]:
df.shape

(20000, 88)

In [4]:
df.columns

Index(['Unnamed: 0', 'Flow.ID', 'Source.IP', 'Source.Port', 'Destination.IP',
       'Destination.Port', 'Protocol', 'Timestamp', 'Flow.Duration',
       'Total.Fwd.Packets', 'Total.Backward.Packets',
       'Total.Length.of.Fwd.Packets', 'Total.Length.of.Bwd.Packets',
       'Fwd.Packet.Length.Max', 'Fwd.Packet.Length.Min',
       'Fwd.Packet.Length.Mean', 'Fwd.Packet.Length.Std',
       'Bwd.Packet.Length.Max', 'Bwd.Packet.Length.Min',
       'Bwd.Packet.Length.Mean', 'Bwd.Packet.Length.Std', 'Flow.Bytes.s',
       'Flow.Packets.s', 'Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max',
       'Flow.IAT.Min', 'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std',
       'Fwd.IAT.Max', 'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean',
       'Bwd.IAT.Std', 'Bwd.IAT.Max', 'Bwd.IAT.Min', 'Fwd.PSH.Flags',
       'Bwd.PSH.Flags', 'Fwd.URG.Flags', 'Bwd.URG.Flags', 'Fwd.Header.Length',
       'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s',
       'Min.Packet.Length', 'Max.Packet.Length', 'Packet.Le

In [5]:
df.dropna(inplace = True) 

In [6]:
df['ProtocolName'].unique()
print(df['ProtocolName'].nunique())

10


In [7]:
pd.set_option("display.max_rows", None)
df['ProtocolName'].value_counts()

SSL               2000
HTTP_CONNECT      2000
GOOGLE            2000
WINDOWS_UPDATE    2000
AMAZON            2000
GMAIL             2000
HTTP              2000
MICROSOFT         2000
YOUTUBE           2000
HTTP_PROXY        2000
Name: ProtocolName, dtype: int64

In [8]:
df_feature = df.copy()
df_feature = df_feature.drop(columns =["Flow.ID", "Timestamp", "Label","ProtocolName" ,"Source.IP", "Source.Port", "Destination.IP", "Destination.Port"], axis = 1)
# df_target = pd.get_dummies(df['ProtocolName'])
df_target = df['ProtocolName']


In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(df_feature)
y = df_target

In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [12]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
y_pred = rf.predict(X_test)

In [14]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

        AMAZON       0.94      0.93      0.94       518
         GMAIL       0.88      0.87      0.88       496
        GOOGLE       0.81      0.85      0.83       450
          HTTP       0.98      0.95      0.96       485
  HTTP_CONNECT       0.85      0.90      0.88       518
    HTTP_PROXY       0.87      0.90      0.89       476
     MICROSOFT       0.98      0.95      0.97       501
           SSL       0.93      0.95      0.94       526
WINDOWS_UPDATE       0.94      0.92      0.93       516
       YOUTUBE       0.88      0.85      0.86       514

      accuracy                           0.91      5000
     macro avg       0.91      0.91      0.91      5000
  weighted avg       0.91      0.91      0.91      5000



In [37]:
# y_true = y_test.idxmax(axis='columns').reset_index().drop('index', axis=1).rename(columns={'0':'ProtocolName'})

In [16]:
# y_pred = pd.DataFrame(pd.DataFrame(rf.predict(X_test), columns = y_test.columns).idxmax(axis='columns'), columns=['ProtocolName'])

In [17]:
# from sklearn.metrics import confusion_matrix

# conf_mat = pd.DataFrame(confusion_matrix(y_true, y_pred,
#                                          labels = df_target.columns),
#                         columns = df_target.columns,
#                         index = df_target.columns)

In [18]:
# print(classification_report(y_true, y_pred))

In [19]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

mlp.fit(X, y)

C:\anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [20]:
y_pred_mlp = mlp.predict(X_test)

In [24]:
print(classification_report(y_test, y_pred_mlp))

                precision    recall  f1-score   support

        AMAZON       1.00      1.00      1.00       518
         GMAIL       1.00      1.00      1.00       496
        GOOGLE       1.00      1.00      1.00       450
          HTTP       1.00      1.00      1.00       485
  HTTP_CONNECT       1.00      1.00      1.00       518
    HTTP_PROXY       1.00      1.00      1.00       476
     MICROSOFT       1.00      1.00      1.00       501
           SSL       1.00      1.00      1.00       526
WINDOWS_UPDATE       1.00      1.00      1.00       516
       YOUTUBE       1.00      1.00      1.00       514

      accuracy                           1.00      5000
     macro avg       1.00      1.00      1.00      5000
  weighted avg       1.00      1.00      1.00      5000



In [29]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [30]:
y_pred_svm = svm.predict(X_test)

In [31]:
print(classification_report(y_test, y_pred_svm))

                precision    recall  f1-score   support

        AMAZON       1.00      1.00      1.00       518
         GMAIL       0.51      0.29      0.37       496
        GOOGLE       0.38      0.46      0.42       450
          HTTP       1.00      1.00      1.00       485
  HTTP_CONNECT       0.45      0.69      0.54       518
    HTTP_PROXY       0.48      0.54      0.51       476
     MICROSOFT       1.00      1.00      1.00       501
           SSL       1.00      1.00      1.00       526
WINDOWS_UPDATE       0.99      0.98      0.99       516
       YOUTUBE       0.56      0.32      0.41       514

      accuracy                           0.73      5000
     macro avg       0.74      0.73      0.72      5000
  weighted avg       0.74      0.73      0.73      5000



In [25]:
from sklearn.ensemble import GradientBoostingClassifier

sgb = GradientBoostingClassifier(n_estimators=100,
                                 learning_rate=1.0,
                                 max_depth=1,
                                 random_state=0)
sgb.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=1.0, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [26]:
y_pred_sgb = sgb.predict(X_test)

In [27]:
print(classification_report(y_test, y_pred_sgb))

                precision    recall  f1-score   support

        AMAZON       1.00      1.00      1.00       518
         GMAIL       1.00      1.00      1.00       496
        GOOGLE       1.00      1.00      1.00       450
          HTTP       1.00      1.00      1.00       485
  HTTP_CONNECT       1.00      1.00      1.00       518
    HTTP_PROXY       1.00      1.00      1.00       476
     MICROSOFT       1.00      1.00      1.00       501
           SSL       1.00      1.00      1.00       526
WINDOWS_UPDATE       1.00      1.00      1.00       516
       YOUTUBE       1.00      1.00      1.00       514

      accuracy                           1.00      5000
     macro avg       1.00      1.00      1.00      5000
  weighted avg       1.00      1.00      1.00      5000



In [34]:
from xgboost import XGBClassifier

xgb = XGBClassifier()

xgb.fit(X, y)

[13:17:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [35]:
y_pred_xgb = xgb.predict(X_test)

In [36]:
print(classification_report(y_test, y_pred_xgb))

                precision    recall  f1-score   support

        AMAZON       1.00      1.00      1.00       518
         GMAIL       1.00      1.00      1.00       496
        GOOGLE       1.00      1.00      1.00       450
          HTTP       1.00      1.00      1.00       485
  HTTP_CONNECT       1.00      1.00      1.00       518
    HTTP_PROXY       1.00      1.00      1.00       476
     MICROSOFT       1.00      1.00      1.00       501
           SSL       1.00      1.00      1.00       526
WINDOWS_UPDATE       1.00      1.00      1.00       516
       YOUTUBE       1.00      1.00      1.00       514

      accuracy                           1.00      5000
     macro avg       1.00      1.00      1.00      5000
  weighted avg       1.00      1.00      1.00      5000

